# Generate Hal imaging profile (and corresponding shutter file)

By Pu Zheng

2023.07.26

In [12]:
import os, sys
import numpy as np

from xml.etree import ElementTree
from xml.dom import minidom
from xml.etree.ElementTree import Element, SubElement, Comment, indent
def prettify(elem):
    """Return a pretty-printed XML string for the Element.
    """
    rough_string = ElementTree.tostring(elem, encoding="ISO-8859-1")
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="\t")

## Default setting of MerScope01

In [13]:
all_channels = [748, 637, 545, 518, 477, 446, 405]


## Select channels here

In [23]:
# required parameters RNA
#sel_channel_ids = [0, 1, 2, 4, 6]
#sel_channel_ids = [0, 1, 2, 4]
sel_channel_ids = [0, 1, 4]
num_steps = 13
step_size = 1.0

sel_channels = [all_channels[_ich] for _ich in sel_channel_ids]
print(f"Selected channels: {sel_channels}")
num_channels = len(sel_channel_ids)
print(f"Num of steps: {num_steps}; step size: {step_size}\u03BCm")

Selected channels: [748, 637, 477]
Num of steps: 13; step size: 1.0μm


# Generate shutter xml

In [24]:
shutter_xml = Element('repeat')
## Add general comments
# comments
comment = Comment(f"channels_{'_'.join([str(_ch) for _ch in sel_channels])}, {num_steps}_frames, {step_size}_each_step")
shutter_xml.append(comment)
# oversample
oversampling = SubElement(shutter_xml, 'oversampling')
oversampling.text = f"{1}"
# flow imaging buffer
num_frames = SubElement(shutter_xml, 'frames')
num_frames.text = f"{num_steps*num_channels}"

## loop through channels

In [25]:
for _i  in np.arange(num_steps):
    for _j, _ch_id in enumerate(sel_channel_ids):
        # determine start time and end time
        _start_time = _i * num_channels + _j
        _end_time = _i * num_channels + _j + 1
        if _ch_id < 0:
            continue
        frame = SubElement(shutter_xml, 'event')
        channel = SubElement(frame, 'channel')
        channel.text = f"{_ch_id}"
        power = SubElement(frame, 'power')
        power.text = f"{1}"
        on = SubElement(frame, 'on')
        on.text = f"{_start_time:.1f}"
        off = SubElement(frame, 'off')
        off.text = f"{_end_time:.1f}"
        color = SubElement(frame, 'color')
        color.text = f"255,255,255"        

In [26]:
full_shutter_str = prettify(shutter_xml)
    
print( full_shutter_str )

<?xml version="1.0" ?>
<repeat>
	<!--channels_748_637_477, 13_frames, 1.0_each_step-->
	<oversampling>1</oversampling>
	<frames>39</frames>
	<event>
		<channel>0</channel>
		<power>1</power>
		<on>0.0</on>
		<off>1.0</off>
		<color>255,255,255</color>
	</event>
	<event>
		<channel>1</channel>
		<power>1</power>
		<on>1.0</on>
		<off>2.0</off>
		<color>255,255,255</color>
	</event>
	<event>
		<channel>4</channel>
		<power>1</power>
		<on>2.0</on>
		<off>3.0</off>
		<color>255,255,255</color>
	</event>
	<event>
		<channel>0</channel>
		<power>1</power>
		<on>3.0</on>
		<off>4.0</off>
		<color>255,255,255</color>
	</event>
	<event>
		<channel>1</channel>
		<power>1</power>
		<on>4.0</on>
		<off>5.0</off>
		<color>255,255,255</color>
	</event>
	<event>
		<channel>4</channel>
		<power>1</power>
		<on>5.0</on>
		<off>6.0</off>
		<color>255,255,255</color>
	</event>
	<event>
		<channel>0</channel>
		<power>1</power>
		<on>6.0</on>
		<off>7.0</off>
		<color>255,255,255</color>
	</event>
	<even

## save shutter xml

In [27]:
parent_folder = r'..\..\example_scripts\hal'
save_folder = os.path.join(parent_folder, f'{num_steps}_steps_n{int(step_size*1000)}')
if not os.path.exists(save_folder):
    os.makedirs(save_folder)
    print(f"creating save folder: {save_folder}")
shutter_basename = "shutter_" + '_'.join([str(_ch) for _ch in sel_channels]) + f'_s{num_steps}.xml'
shutter_filename = os.path.join(save_folder, shutter_basename)
overwrite = False
if not os.path.isfile(shutter_filename) or overwrite:
    print(f"-- save the shutter to file: {shutter_filename}")
    # write
    with open(shutter_filename, 'w') as _fp:
        _fp.write(full_shutter_str)
else:
    print(f"-- shutter file: {shutter_filename} already exists, skip")

-- shutter file: ..\..\example_scripts\hal\13_steps_n1000\shutter_748_637_477_s13.xml already exists, skip


# Generate hal profile

## Default parameters

In [28]:
_frame_rate = 5 # Hz
_log_folder = r'E:\Data\logs'
_default_power = 0.1
low_power = 0.02
start_z = -1 * num_steps * step_size / 2


## generate hal xml

In [29]:
hal_xml = Element('settings')

# Camera
hal_xml.append(Comment(f"camera settings"))
camera1 = SubElement(hal_xml, 'camera1')

camera1.append(Comment(f"exposure time"))
exposure_time = SubElement(camera1, 'exposure_time', type='float')
exposure_time.text=f'{1/_frame_rate:.2f}'

camera1.append(Comment(f"view settings"))
flip_horizontal = SubElement(camera1, 'flip_horizontal', type='int')
flip_horizontal.text=f'{0}'
flip_vertical = SubElement(camera1, 'flip_vertical', type='int')
flip_vertical.text=f'{0}'
transpose = SubElement(camera1, 'transpose', type='int')
transpose.text=f'{0}'
scalemax = SubElement(camera1, 'scalemax', type='int')
scalemax.text=f'{20000}'
scalemin = SubElement(camera1, 'scalemin', type='int')
scalemin.text=f'{200}'
max_intensity = SubElement(camera1, 'max_intensity', type='int')
max_intensity.text=f'{65536}'
colortable = SubElement(camera1, 'colortable', type='string')
colortable.text=f'ramp_sat.ctbl'
sync = SubElement(camera1, 'sync', type='int')
sync.text=f'{0}'

# film
hal_xml.append(Comment(f"film settings"))
film = SubElement(hal_xml, 'film')
filename = SubElement(film, 'filename', type='string')
filename.text = f"zscan_{'_'.join([str(_ch) for _ch in sel_channels])}_s{num_steps}_n{int(step_size*1000)}_{int(_frame_rate)}Hz"
acq_mode = SubElement(film, 'acq_mode', type='string')
acq_mode.text = f"fixed_length"
frames = SubElement(film, 'frames', type='int')
frames.text = f"{num_steps*num_channels}"
logfile = SubElement(film, 'logfile', type='string')
logfile.text = f"{os.path.join(_log_folder, 'hal_log.txt')}"
auto_increment = SubElement(film, 'auto_increment', type='int')
auto_increment.text=f'{1}'
auto_shutters = SubElement(film, 'auto_shutters', type='int')
auto_shutters.text=f'{1}'
want_bell = SubElement(film, 'want_bell', type='int')
want_bell.text=f'{1}'
filetype = SubElement(film, 'filetype', type='string')
filetype.text = f".dax"

# illumination
hal_xml.append(Comment(f"illumination settings"))
illumination = SubElement(hal_xml, 'illumination')
default_power = SubElement(illumination, 'default_power', type='custom')
_default_power_list = []
for _ch_id, _ch in enumerate(all_channels):
    if _ch_id in sel_channel_ids:
        _default_power_list.append(f"{_default_power:.2f}")
    else:
        _default_power_list.append(f"{0:.2f}")
default_power.text = f"{','.join(_default_power_list)}"
shutters = SubElement(illumination, 'shutters', type='string')
shutters.text = f"{shutter_basename}"
on_off_state = SubElement(illumination, 'on_off_state', type='custom')
on_off_state.text = ",".join(["False" for _i in range(len(all_channels))])
power_buttons = SubElement(illumination, 'power_buttons', type='custom')
power_buttons.text = str([[['Max', 1.0], ['Low', low_power]] for _i in range(len(all_channels))])

# focuslock
hal_xml.append(Comment(f"focus lock settings"))
focuslock = SubElement(hal_xml, 'focuslock')
locked = SubElement(focuslock, 'locked')
buffer_length = SubElement(locked, 'buffer_length', type='int')
buffer_length.text=f"{5}"
lock_gain = SubElement(locked, 'lock_gain', type='float')
lock_gain.text=f"{0.1}"
lock_gain_max = SubElement(locked, 'lock_gain_max', type='float')
lock_gain_max.text=f"{0.1}"
minimum_sum = SubElement(locked, 'minimum_sum', type='float')
minimum_sum.text=f"{-1.0:.1f}"
offset_threshold = SubElement(locked, 'offset_threshold', type='float')
offset_threshold.text=f"{50.0:.1f}"
hardware_z_scan = SubElement(focuslock, 'hardware_z_scan')
z_offsets = SubElement(hardware_z_scan, 'z_offsets', type='string')
offset_string = []
for _i in np.arange(num_steps):
    offset_string.extend([f"{round(start_z+_i*step_size, 2):.2f}"]*num_channels)
z_offsets.text = ','.join(offset_string)

In [30]:
full_hal_str = prettify(hal_xml)
    
print( full_hal_str )

<?xml version="1.0" ?>
<settings>
	<!--camera settings-->
	<camera1>
		<!--exposure time-->
		<exposure_time type="float">0.20</exposure_time>
		<!--view settings-->
		<flip_horizontal type="int">0</flip_horizontal>
		<flip_vertical type="int">0</flip_vertical>
		<transpose type="int">0</transpose>
		<scalemax type="int">20000</scalemax>
		<scalemin type="int">200</scalemin>
		<max_intensity type="int">65536</max_intensity>
		<colortable type="string">ramp_sat.ctbl</colortable>
		<sync type="int">0</sync>
	</camera1>
	<!--film settings-->
	<film>
		<filename type="string">zscan_748_637_477_s13_n1000_5Hz</filename>
		<acq_mode type="string">fixed_length</acq_mode>
		<frames type="int">39</frames>
		<logfile type="string">E:\Data\logs\hal_log.txt</logfile>
		<auto_increment type="int">1</auto_increment>
		<auto_shutters type="int">1</auto_shutters>
		<want_bell type="int">1</want_bell>
		<filetype type="string">.dax</filetype>
	</film>
	<!--illumination settings-->
	<illumination>
		<def

## save hal xml

In [31]:
parent_folder = r'..\..\example_scripts\hal'
save_folder = os.path.join(parent_folder, f'{num_steps}_steps_n{int(step_size*1000)}')
if not os.path.exists(save_folder):
    os.makedirs(save_folder)
    print(f"creating save folder: {save_folder}")
hal_basename = f"Conv_zscan_{'_'.join([str(_ch) for _ch in sel_channels])}_s{num_steps}_n{int(step_size*1000)}_{int(_frame_rate)}Hz.xml"
hal_filename = os.path.join(save_folder, hal_basename)
overwrite = False
if not os.path.isfile(hal_filename) or overwrite:
    print(f"-- save the hal to file: {hal_filename}")
    # write
    with open(hal_filename, 'w') as _fp:
        _fp.write(full_hal_str)
else:
    print(f"-- hal file: {hal_filename} already exists, skip")

-- save the hal to file: ..\..\example_scripts\hal\13_steps_n1000\Conv_zscan_748_637_477_s13_n1000_5Hz.xml


In [40]:
# copy the output of this box into the hal setting:
